# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
weather_csv = "weather_data.csv"

# Read the mouse data and the study results
weather_df = pd.read_csv(weather_csv)
weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Cherskiy,0,RU,1580528897,98,68.75,161.30,-29.58,4.12
1,1,Changtu,0,CN,1580528899,81,42.78,124.10,11.73,4.97
2,2,Khatanga,0,RU,1580528900,97,71.97,102.50,-33.97,6.82
3,3,Saskylakh,100,RU,1580528901,97,71.92,114.08,-24.20,8.66
4,4,Kaitangata,100,NZ,1580528644,65,-46.28,169.85,65.52,22.28
...,...,...,...,...,...,...,...,...,...,...
562,562,Shenjiamen,0,CN,1580529719,59,29.96,122.30,46.44,5.41
563,563,Parana,0,AR,1580529675,47,-31.73,-60.52,75.00,1.01
564,564,Riyadh,29,SA,1580529568,30,24.69,46.72,59.00,6.93
565,565,Padang,78,ID,1580529360,65,-0.95,100.35,79.90,4.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = weather_df[["Lat", "Lng"]].astype(float)
weather_data_complete = weather_df.dropna()
weight = weather_df["Humidity"].astype(float)


In [24]:
# Add Heatmap layer to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 3, opacity=0.6)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
weather_new_df = weather_df.drop('Unnamed: 0', 1)
weather_new_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Cherskiy,0,RU,1580528897,98,68.75,161.30,-29.58,4.12
1,Changtu,0,CN,1580528899,81,42.78,124.10,11.73,4.97
2,Khatanga,0,RU,1580528900,97,71.97,102.50,-33.97,6.82
3,Saskylakh,100,RU,1580528901,97,71.92,114.08,-24.20,8.66
4,Kaitangata,100,NZ,1580528644,65,-46.28,169.85,65.52,22.28
...,...,...,...,...,...,...,...,...,...
562,Shenjiamen,0,CN,1580529719,59,29.96,122.30,46.44,5.41
563,Parana,0,AR,1580529675,47,-31.73,-60.52,75.00,1.01
564,Riyadh,29,SA,1580529568,30,24.69,46.72,59.00,6.93
565,Padang,78,ID,1580529360,65,-0.95,100.35,79.90,4.36


In [21]:
good_temperatures_df = weather_new_df.loc[(weather_new_df["Max Temp"] >= 59)&(weather_new_df["Max Temp"] <= 77), 
                                          ["City", "Cloudiness", "Country", "Date", "Humidity", 
                                           "Lat", "Lng", "Max Temp", "Wind Speed"]]
good_temperatures_df.sort_values("Max Temp", ascending=False)
good_cloudiness_df = good_temperatures_df.loc[good_temperatures_df["Cloudiness"] <50, ["City", "Cloudiness", "Country", "Date", "Humidity", 
                                           "Lat", "Lng", "Max Temp", "Wind Speed"]]
good_cloudiness_df
good_wind_speed_df = good_cloudiness_df.loc[good_cloudiness_df["Wind Speed"]<20,["City", "Cloudiness", "Country", "Date", "Humidity", 
                                           "Lat", "Lng", "Max Temp", "Wind Speed"]]
good_wind_speed_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
23,Correntina,0,BR,1580528930,79,-13.34,-44.64,66.47,1.32
29,Bredasdorp,22,ZA,1580528939,77,-34.53,20.04,66.20,4.70
35,Lompoc,1,US,1580528948,54,34.64,-120.46,60.80,4.90
37,Wad Medani,0,SD,1580528951,25,14.40,33.52,59.85,10.40
69,Arraial do Cabo,40,BR,1580528997,94,-22.97,-42.03,75.20,4.70
...,...,...,...,...,...,...,...,...,...
547,Moron,0,AR,1580529696,53,-34.65,-58.62,75.20,4.70
550,Mankachar,0,IN,1580529701,50,25.53,89.87,67.75,3.56
555,Harlingen,40,US,1580529603,81,26.19,-97.70,60.80,3.36
563,Parana,0,AR,1580529675,47,-31.73,-60.52,75.00,1.01


In [43]:
good_wind_speed_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
23,Correntina,0,BR,1580528930,79,-13.34,-44.64,66.47,1.32
29,Bredasdorp,22,ZA,1580528939,77,-34.53,20.04,66.20,4.70
35,Lompoc,1,US,1580528948,54,34.64,-120.46,60.80,4.90
37,Wad Medani,0,SD,1580528951,25,14.40,33.52,59.85,10.40
69,Arraial do Cabo,40,BR,1580528997,94,-22.97,-42.03,75.20,4.70
...,...,...,...,...,...,...,...,...,...
547,Moron,0,AR,1580529696,53,-34.65,-58.62,75.20,4.70
550,Mankachar,0,IN,1580529701,50,25.53,89.87,67.75,3.56
555,Harlingen,40,US,1580529603,81,26.19,-97.70,60.80,3.36
563,Parana,0,AR,1580529675,47,-31.73,-60.52,75.00,1.01


In [62]:
pre_best_vacation_df=good_wind_speed_df.loc[good_wind_speed_df["Lng"]<-30,["City", "Cloudiness", "Country", "Date", "Humidity", 
                                           "Lat", "Lng", "Max Temp", "Wind Speed"]]
pre_best_vacation_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
23,Correntina,0,BR,1580528930,79,-13.34,-44.64,66.47,1.32
35,Lompoc,1,US,1580528948,54,34.64,-120.46,60.80,4.90
69,Arraial do Cabo,40,BR,1580528997,94,-22.97,-42.03,75.20,4.70
94,Florida,1,US,1580528804,100,28.75,-82.50,62.01,2.42
110,Castro,1,BR,1580529059,90,-24.79,-50.01,66.88,6.11
117,Chui,0,UY,1580529070,82,-33.70,-53.46,64.81,5.64
159,Lima,40,PE,1580528844,83,-12.04,-77.03,75.20,14.99
165,San Francisco,40,US,1580528709,87,37.77,-122.42,60.80,5.14
172,Matamoros,40,MX,1580529149,23,25.53,-103.25,60.80,6.82
179,Rio Grande,3,BR,1580529063,80,-32.03,-52.10,71.33,7.31


In [65]:
best_vacation_df=pre_best_vacation_df.loc[(pre_best_vacation_df["Country"]=="US")|(pre_best_vacation_df["Country"]=="MX"),["City", "Cloudiness", "Country", "Date", "Humidity", 
                                           "Lat", "Lng", "Max Temp", "Wind Speed"]]
best_vacation_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
35,Lompoc,1,US,1580528948,54,34.64,-120.46,60.80,4.90
94,Florida,1,US,1580528804,100,28.75,-82.50,62.01,2.42
165,San Francisco,40,US,1580528709,87,37.77,-122.42,60.80,5.14
172,Matamoros,40,MX,1580529149,23,25.53,-103.25,60.80,6.82
203,Caborca,0,MX,1580529195,35,30.58,-111.10,66.99,3.00
232,Pacific Grove,1,US,1580529237,76,36.62,-121.92,60.01,3.36
341,San Quintin,0,MX,1580529398,47,30.48,-115.95,62.04,6.44
555,Harlingen,40,US,1580529603,81,26.19,-97.70,60.80,3.36


In [66]:
good_locations = best_vacation_df[["Lat", "Lng"]].astype(float)
good_weight = best_vacation_df["Max Temp"].astype(float)

In [73]:
fig = gmaps.figure()
good_heat_layer = gmaps.heatmap_layer(good_locations, weights=good_weight, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 3, opacity=0.6)

fig.add_layer(good_heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [68]:
hotel_df = best_vacation_df
hotel_df['Hotel Name'] = np.nan

target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1

narrowed_city_df = hotel_df
narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
35,Lompoc,1,US,1580528948,54,34.64,-120.46,60.80,4.90,Embassy Suites by Hilton Lompoc Central Coast
94,Florida,1,US,1580528804,100,28.75,-82.50,62.01,2.42,NaN
165,San Francisco,40,US,1580528709,87,37.77,-122.42,60.80,5.14,Hyatt Regency San Francisco
172,Matamoros,40,MX,1580529149,23,25.53,-103.25,60.80,6.82,NaN
203,Caborca,0,MX,1580529195,35,30.58,-111.10,66.99,3.00,Motel Elba
232,Pacific Grove,1,US,1580529237,76,36.62,-121.92,60.01,3.36,Monterey Plaza Hotel & Spa
341,San Quintin,0,MX,1580529398,47,30.48,-115.95,62.04,6.44,Old Mill Hotel
555,Harlingen,40,US,1580529603,81,26.19,-97.70,60.80,3.36,Best Western Casa Villa Suites


In [70]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations3 = hotel_df[["Lat", "Lng"]]

In [74]:
# Add marker layer ontop of heat map
vacation_marker_layer = gmaps.marker_layer(locations3, info_box_content=hotel_info)
fig.add_layer(vacation_marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))